# 🚀 Infrastructure Quick Reference — DAARION & MicroDAO

**Версія:** 1.3.0  
**Останнє оновлення:** 2025-01-21  

Цей notebook містить швидкий довідник по серверах, репозиторіях та endpoints для DAGI Stack.

**UPDATED:** Swapper Service (vision-8b) на Node #1, Node #2 (MacBook Pro M4 Max)

**Note:** На Node #1 використовується Swapper Service з vision-8b моделлю (Qwen3-VL 8B) для обробки зображень, а не Vision Encoder.

In [ ]:
# Service Configuration (UPDATED with Swapper Service на Node #1)
SERVICES = {
    "router": {"port": 9102, "container": "dagi-router", "health": "http://localhost:9102/health"},
    "gateway": {"port": 9300, "container": "dagi-gateway", "health": "http://localhost:9300/health"},
    "devtools": {"port": 8008, "container": "dagi-devtools", "health": "http://localhost:8008/health"},
    "crewai": {"port": 9010, "container": "dagi-crewai", "health": "http://localhost:9010/health"},
    "rbac": {"port": 9200, "container": "dagi-rbac", "health": "http://localhost:9200/health"},
    "rag": {"port": 9500, "container": "dagi-rag-service", "health": "http://localhost:9500/health"},
    "memory": {"port": 8000, "container": "dagi-memory-service", "health": "http://localhost:8000/health"},
    "parser": {"port": 9400, "container": "dagi-parser-service", "health": "http://localhost:9400/health"},
    "swapper": {"port": 8890, "container": "swapper-service", "health": "http://localhost:8890/health", "gpu": True, "note": "Dynamic model swapping (vision-8b, math-7b, etc.)"},
    "postgres": {"port": 5432, "container": "dagi-postgres", "health": None},
    "redis": {"port": 6379, "container": "redis", "health": "redis-cli PING"},
    "neo4j": {"port": 7474, "container": "neo4j", "health": "http://localhost:7474"},
    "qdrant": {"port": 6333, "container": "dagi-qdrant", "health": "http://localhost:6333/healthz"},
    "grafana": {"port": 3000, "container": "grafana", "health": "http://localhost:3000"},
    "prometheus": {"port": 9090, "container": "prometheus", "health": "http://localhost:9090"},
    "ollama": {"port": 11434, "container": "ollama", "health": "http://localhost:11434/api/tags"}
}

print("Service\t\t\tPort\tContainer\t\t\tHealth Endpoint")
print("="*100)
for name, service in SERVICES.items():
    health = service['health'] or "N/A"
    gpu = " [GPU]" if service.get('gpu') else ""
    print(f"{name.upper():<20} {service['port']:<7} {service['container']:<30} {health}{gpu}")

## 🖥️ Network Nodes

### Node #1: Production Server (Hetzner)
- **Node ID:** node-1-hetzner-gex44
- **IP:** 144.76.224.179
- **Role:** Production Router + Gateway + All Services (24/7)
- **Location:** Hetzner Cloud (Germany)

### Node #2: Development Node (MacBook Pro M4 Max)
- **Node ID:** node-2-macbook-m4max
- **Local IP:** 192.168.1.244
- **Role:** Development + Testing + Backup Router
- **Specs:** M4 Max (16 cores), 64GB RAM, 2TB SSD, 40-core GPU
- **Location:** Local Network (Ivan's Office)
- **Docs:** [NODE-2-MACBOOK-SPECS.md](../NODE-2-MACBOOK-SPECS.md)

---

In [ ]:
# Network Nodes Configuration
NODES = {
    "node-1": {
        "name": "Hetzner GEX44",
        "ip": "144.76.224.179",
        "local_ip": None,
        "role": "production",
        "uptime": "24/7",
        "ssh": "root@144.76.224.179",
        "domain": "gateway.daarion.city",
        "services": "All (17 services)",
        "specs": "See SYSTEM-INVENTORY.md"
    },
    "node-2": {
        "name": "MacBook Pro M4 Max",
        "ip": None,
        "local_ip": "192.168.1.244",
        "role": "development",
        "uptime": "on-demand",
        "ssh": "apple@192.168.1.244",
        "domain": None,
        "services": "Core only (Router, DevTools, Memory, Ollama)",
        "specs": "M4 Max, 16 cores, 64GB RAM, 2TB SSD, 40-core GPU"
    }
}

print("DAGI Stack Network Nodes:")
print("="*80)
for node_id, node in NODES.items():
    print(f"\n{node_id.upper()}: {node['name']}")
    print(f"  Role:     {node['role']}")
    print(f"  IP:       {node['ip'] or node['local_ip']}")
    print(f"  SSH:      {node['ssh']}")
    print(f"  Uptime:   {node['uptime']}")
    print(f"  Services: {node['services']}")
    if node['domain']:
        print(f"  Domain:   https://{node['domain']}")
    print(f"  Specs:    {node['specs']}")

## 🎨 Vision Encoder Service (NEW)

### Overview
- **Service:** Vision Encoder (OpenCLIP ViT-L/14)
- **Port:** 8001
- **GPU:** Required (NVIDIA CUDA)
- **Embedding Dimension:** 768
- **Vector DB:** Qdrant (port 6333/6334)

In [ ]:
# Vision Encoder Configuration
VISION_ENCODER = {
    "service": "vision-encoder",
    "port": 8001,
    "container": "dagi-vision-encoder",
    "gpu_required": True,
    "model": "ViT-L-14",
    "pretrained": "openai",
    "embedding_dim": 768,
    "endpoints": {
        "health": "http://localhost:8001/health",
        "info": "http://localhost:8001/info",
        "embed_text": "http://localhost:8001/embed/text",
        "embed_image": "http://localhost:8001/embed/image",
        "docs": "http://localhost:8001/docs"
    },
    "qdrant": {
        "host": "qdrant",
        "port": 6333,
        "grpc_port": 6334,
        "health": "http://localhost:6333/healthz"
    }
}

print("Vision Encoder Service Configuration:")
print("="*80)
print(f"Model: {VISION_ENCODER['model']} ({VISION_ENCODER['pretrained']})")
print(f"Embedding Dimension: {VISION_ENCODER['embedding_dim']}")
print(f"GPU Required: {VISION_ENCODER['gpu_required']}")
print(f"\nEndpoints:")
for name, url in VISION_ENCODER['endpoints'].items():
    print(f"  {name:15} {url}")
print(f"\nQdrant Vector DB:")
print(f"  HTTP:  http://localhost:{VISION_ENCODER['qdrant']['port']}")
print(f"  gRPC:  localhost:{VISION_ENCODER['qdrant']['grpc_port']}")

In [ ]:
# Vision Encoder Testing Commands
VISION_ENCODER_TESTS = {
    "Health Check": "curl http://localhost:8001/health",
    "Model Info": "curl http://localhost:8001/info",
    "Text Embedding": '''curl -X POST http://localhost:8001/embed/text -H "Content-Type: application/json" -d '{"text": "DAARION governance", "normalize": true}' ''',
    "Image Embedding": '''curl -X POST http://localhost:8001/embed/image -H "Content-Type: application/json" -d '{"image_url": "https://example.com/image.jpg", "normalize": true}' ''',
    "Via Router (Text)": '''curl -X POST http://localhost:9102/route -H "Content-Type: application/json" -d '{"mode": "vision_embed", "message": "embed text", "payload": {"operation": "embed_text", "text": "test", "normalize": true}}' ''',
    "Qdrant Health": "curl http://localhost:6333/healthz",
    "Run Smoke Tests": "./test-vision-encoder.sh"
}

print("Vision Encoder Testing Commands:")
print("="*80)
for name, cmd in VISION_ENCODER_TESTS.items():
    print(f"\n{name}:")
    print(f"  {cmd}")

## 📖 Documentation Links (UPDATED)

In [ ]:
# Documentation References (UPDATED)
DOCS = {
    "Main Guide": "../WARP.md",
    "Infrastructure": "../INFRASTRUCTURE.md",
    "Agents Map": "../docs/agents.md",
    "RAG Ingestion Status": "../RAG-INGESTION-STATUS.md",
    "HMM Memory Status": "../HMM-MEMORY-STATUS.md",
    "Crawl4AI Status": "../CRAWL4AI-STATUS.md",
    "Vision Encoder Status": "../VISION-ENCODER-STATUS.md",
    "Vision Encoder Deployment": "../services/vision-encoder/README.md",
    "Repository Management": "../DAARION_CITY_REPO.md",
    "Server Setup": "../SERVER_SETUP_INSTRUCTIONS.md",
    "Deployment": "../DEPLOY-NOW.md",
    "Helion Status": "../STATUS-HELION.md",
    "Architecture Index": "../docs/cursor/README.md",
    "API Reference": "../docs/api.md",
    "Node #2 Specs": "../NODE-2-MACBOOK-SPECS.md"
}

print("Documentation Quick Links:")
print("="*80)
for name, path in DOCS.items():
    print(f"{name:<30} {path}")

## 📝 Notes & Updates

### Recent Changes (2025-01-17)
- ✅ **Added Vision Encoder Service** (port 8001) with OpenCLIP ViT-L/14
- ✅ **Added Qdrant Vector Database** (port 6333/6334) for image/text embeddings
- ✅ **GPU Support** via NVIDIA CUDA + Docker runtime
- ✅ **DAGI Router integration** (mode: vision_embed)
- ✅ **768-dim embeddings** for multimodal RAG
- ✅ Created VISION-ENCODER-STATUS.md with full implementation details
- ✅ Added test-vision-encoder.sh smoke tests
- ✅ **Added Node #2** (MacBook Pro M4 Max) as development node
- ✅ Created NODE-2-MACBOOK-SPECS.md with complete specifications

### Network Architecture
- **Nodes:** 2 (1 production + 1 development)
- **Total Services:** 17
- **GPU Services:** 1 (Vision Encoder) + Node #2 (40-core Apple GPU)
- **Vector Databases:** 1 (Qdrant)

---

**Last Updated:** 2025-01-17 by WARP AI  
**Maintained by:** Ivan Tytar & DAARION Team